# Models Training Status

In this notebook there are scripts to check which models are there and which are missing.

In [51]:
base_path = "/home/tbaldi/checkpoint/"
sizes = ['small', 'baseline', 'large']
batch_sizes = [16, 32, 64, 128, 256, 512, 1024]
learning_rates = [0.1, 0.05, 0.025, 0.0125, 0.00625, 0.003125, 0.0015625]
precisions = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [52]:
missing = []
def get_missing_econ(bs, lr, precision):
    missing = []
    for size in sizes:
        print(f'BATCH SIZE: {bs} - LEARNING RATE {lr} - SIZE {size} - PRECISION {precision}')
        for exp in range(1, 4):
            file_path = base_path + f'bs{bs}_lr{lr}/ECON_{precision}b/{size}/{size}_emd_{exp}.txt'
            try:
                with open(file_path, 'r') as file:
                    content = file.read()

            except Exception as e:
                if size == 'small':
                    print(file_path)
                    missing.append((bs, lr, size))
                    continue
    # missing = sorted(missing, key=lambda x: x[2])
    return missing


In [53]:
for size in sizes:
    print(size, get_missing_econ(1024, 0.1, 2))

BATCH SIZE: 1024 - LEARNING RATE 0.1 - SIZE small - PRECISION 2
/home/tbaldi/checkpoint/bs1024_lr0.1/ECON_2b/small/small_emd_1.txt
/home/tbaldi/checkpoint/bs1024_lr0.1/ECON_2b/small/small_emd_2.txt
/home/tbaldi/checkpoint/bs1024_lr0.1/ECON_2b/small/small_emd_3.txt
BATCH SIZE: 1024 - LEARNING RATE 0.1 - SIZE baseline - PRECISION 2
BATCH SIZE: 1024 - LEARNING RATE 0.1 - SIZE large - PRECISION 2
small [(1024, 0.1, 'small'), (1024, 0.1, 'small'), (1024, 0.1, 'small')]
BATCH SIZE: 1024 - LEARNING RATE 0.1 - SIZE small - PRECISION 2
/home/tbaldi/checkpoint/bs1024_lr0.1/ECON_2b/small/small_emd_1.txt
/home/tbaldi/checkpoint/bs1024_lr0.1/ECON_2b/small/small_emd_2.txt
/home/tbaldi/checkpoint/bs1024_lr0.1/ECON_2b/small/small_emd_3.txt
BATCH SIZE: 1024 - LEARNING RATE 0.1 - SIZE baseline - PRECISION 2
BATCH SIZE: 1024 - LEARNING RATE 0.1 - SIZE large - PRECISION 2
baseline [(1024, 0.1, 'small'), (1024, 0.1, 'small'), (1024, 0.1, 'small')]
BATCH SIZE: 1024 - LEARNING RATE 0.1 - SIZE small - PRECISI

In [54]:
for tmp in set(sorted_missing):
    if tmp[0] == 512:
        print(f'{tmp[0]}\t\t{tmp[1]}\t\t{tmp[2]}')

512		0.00625		small
512		0.003125		small
512		0.05		small
512		0.1		small
512		0.0015625		small
512		0.025		small
512		0.0125		small


In [55]:

for tmp in set(sorted_missing):
    if tmp[0] == 256:
        print(tmp[0], tmp[1], tmp[2])

256 0.05 small
256 0.1 small
256 0.0015625 small
256 0.025 small
256 0.0125 small
256 0.00625 small
256 0.003125 small


In [56]:
for tmp in set(sorted_missing):
    if tmp[0] == 128:
        print(tmp[0], tmp[1], tmp[2])

128 0.0125 small
128 0.00625 small
128 0.003125 small
128 0.05 small
128 0.1 small
128 0.025 small
128 0.0015625 small


In [57]:
for tmp in set(sorted_missing):
    if tmp[0] == 64:
        print(tmp[0], tmp[1], tmp[2])

64 0.0015625 small
64 0.025 small
64 0.0125 small
64 0.00625 small
64 0.003125 small
64 0.05 small
64 0.1 small


In [58]:
for tmp in set(sorted_missing):
    if tmp[0] == 32:
        print(tmp[0], tmp[1], tmp[2])

32 0.05 small
32 0.1 small
32 0.025 small
32 0.0015625 small
32 0.0125 small
32 0.00625 small
32 0.003125 small


In [59]:
for tmp in set(sorted_missing):
    if tmp[0] == 16:
        print(tmp[0], tmp[1], tmp[2])

16 0.0125 small
16 0.00625 small
16 0.003125 small
16 0.05 small
16 0.1 small
16 0.0015625 small
16 0.025 small
